# Michael Navarro: Competency 2 - Project 2

In [ ]:
#----------------------------------------------------------------------------------------
#                                       Imports 
#----------------------------------------------------------------------------------------
import os
import pandas as pd
import numpy as np
from scipy import stats
from pandas.plotting import scatter_matrix
from pandas.plotting import lag_plot

#----------------------------------------------------------------------------------------
#                                   Global Constants
#----------------------------------------------------------------------------------------
HOME_DIR = os.getcwd()
APP_DATA_FNAME = "Application_Data.csv"

In [ ]:
# Load application file data
app_data = pd.read_csv(APP_DATA_FNAME)

## Requirement #01: Look At Records
In the following code result block we can see the first 10 results. Looking at the attributes we can see that they are either numerical or categorical. Luckily this is a very clean dataset in that none of the attributes are made up of a list of values.

In [ ]:
app_data.head(10)

## Requirement #02: Display Statistical Information

In [ ]:
print(f'Number of Rows: {len(app_data)}')
print(f'Number of Cols: {len(app_data.columns)}')

In [ ]:
app_data.describe()

## Requirement #03: Data Exploration

First lets see if there are any null values in the dataframe by using the info() method. As we can see in the following code result output, there does not appear to be any attributes with a null value.

In [ ]:
app_data.info()

Just to be sure, let us make sure that there are no attributes with a null value.

In [ ]:
# Count the number of attributes that are null
num_nulls = app_data.isna().sum().sum()

print(f'Nuber of NULL values: {num_nulls}')

Now that we have ensured that there are no NULL values, let us enumerate the categorical attributes so that we can have some more numerical attributes to use as features.

In [ ]:
# Factorize/Enumerate text cols for arithmetic
app_data['Gender_Code']     = app_data.Applicant_Gender.factorize()[0]
app_data['Income_Code']     = app_data.Income_Type.factorize()[0]
app_data['Education_Code']  = app_data.Education_Type.factorize()[0]
app_data['Family_Code']     = app_data.Family_Status.factorize()[0]
app_data['Housing_Code']    = app_data.Housing_Type.factorize()[0]
app_data['Job_Code']        = app_data.Job_Title.factorize()[0]

Now that we have added some new numerical attributes representing categorical attributes, let's see look at their statistical information.

In [ ]:
# Save enumerated attributes into a list
enumed_attributes = ['Gender_Code', 'Income_Code', 'Education_Code', 'Family_Code', 'Housing_Code', 'Job_Code']

# Create a new datafrom with just the enumerated attributes
enumed_df = app_data[enumed_attributes]

# Show statistical info
enumed_df.describe()

Before we proceed any further with the data exploration portion, let us decide what the features, and label will be.

This dataset is composed of application data for credit cards. Considering the end goal of such an application is to get a credit card, the label in this case will be the Status. The status coloumn is a binary numerical value. The number "1" indicates the application was succesfull, while the number "0" indicates a failure.

The features in this case should be related to the label, that is, the result of the application. Unfortunately, I do not know which attributes are actually taken into consideration when applying for a credit card. However, from personal experience working in retail and seeing the questions asked, I can take a guess.

In order to not miss any kind of relationships between the features and the label, I will save one list with all attributes except "Status" and another with cherry picked attributes from personal experience.

In [ ]:
# Save numerical columns into a python list
all_features = list(app_data.select_dtypes([np.number]).columns)
# Remove Applicant ID, this is an abstract attribute used as a unique row ID identifier in databases if no Primary Key is created.
all_features.remove('Applicant_ID')
# Remove our label
all_features.remove('Status')
# Remove Owned Mobile Phone because all values in this column are 1
all_features.remove('Owned_Mobile_Phone')

# Save selected numerical columns as features
sel_features = ['Owned_Car', 'Owned_Realty', 'Total_Income', 'Total_Family_Members',
                'Applicant_Age', 'Total_Bad_Debt', 'Total_Good_Debt', 'Gender_Code', 
                'Family_Code', 'Housing_Code', 'Job_Code']

# Uncomment to see the result of the variables
# print('===================== ALL ATTRIBUTES =====================')
# print(all_features)
# print()
# print('===================== SEL ATTRIBUTES =====================')
# print(sel_features)

Now that we have our features sorted out, let us remove any rows with outliers to normalize our data.

In [ ]:
# Make a copy of sel features because we are going to need to remove Total_Income before plotting
tmp_feats = all_features.copy()

# Normalize the data by removing anything above 2 std deviations using the z-score of each respective column
# I chose 2 std deviations becuase this is what I was taught in my statistics class is normal.
# We can switch to three if we want.
# 2 std deviations = 95% of data
# 3 std deviations = 99.7% of data
std_dev = 2
normalized_data_all = app_data[(np.abs(stats.zscore(app_data[tmp_feats])) < std_dev).all(axis=1)]

# Remove Total_Income because it ruins the graph scaling.
tmp_feats.remove('Total_Income')

In [ ]:
# Plot all selected features except Total_Income
normalized_data_all.boxplot(column = tmp_feats, figsize=(40, 40))

In [ ]:
normalized_data_all.boxplot(column = ['Total_Income'], figsize=(40, 40))

In [ ]:
# Make a copy of sel features because we are going to need to remove Total_Income before plotting
tmp_feats = sel_features.copy()

# Normalize the data by removing anything above 2 std deviations using the z-score of each respective column
normalized_data_sel = app_data[(np.abs(stats.zscore(app_data[tmp_feats])) < std_dev).all(axis=1)]

# Remove Total_Income because it ruins the graph scaling.
tmp_feats.remove('Total_Income')

In [ ]:
# Plot all selected features except Total_Income
normalized_data_sel.boxplot(column = tmp_feats, figsize=(40, 40))

In [ ]:
normalized_data_sel.boxplot(column = ['Total_Income'], figsize=(40, 40))

The following scatter matrix for all attributes will serve as an eagle eye view to see if any kind of relationship jumps out at us/

### !----------------- THE FOLLOWING CODE BLOCK TAKES A WHILE AND PRODUCES A LARGE IMAGE --------------------!
For readabilites and speesd sake, it will be commented out, please uncomment it if you would like to see it.

In [ ]:
scatter_matrix(normalized_data_all[all_features], figsize=(40, 40))
print() # This print is only used to stop the spew from scatter_matrix

Now, let us take a look at the hand selected features.

In [ ]:
scatter_matrix(normalized_data_sel[sel_features], figsize=(40, 40))
print() # This print is only used to stop the spew from scatter_matrix

### !--------------------------------------------------------------------------------------------------------------------------------!

Let us take a look at a couple more plots to see if which list of features may be better.

In [ ]:
print('ALL FEATURES')
lag_plot(normalized_data_all[all_features])
print()

In [ ]:
print('SELECTED FEATURES')
lag_plot(normalized_data_sel[sel_features])
print()

Looking at both the lag plots, it looks like out chosen selected features match. The fact that there is a discernable structure to the lag plot tells us that there is some sort of relationship that can lead us to predicting which applications will be successful and which won't.

## Extras

### Augmentation
It is possible to augment this data. We could get the data from another source and add it to ours. This would be an example of real data augemtnation. Another way to ad data would be to do what statisticaians do and use the standard deviation mean to create records within the avergae bounds of the dataset.

It may not be needed though, we did manage to see that the data had an overwhelming majority of successful applications versus unsuccessful applications. This may be a sign of bias that our dataset may be unusable. For exmaple, if the dataset comes from a high end department store, then the common demographic that frequents the store will have an advantage. 

However, if the department store itself is requesting this research, then the dataset may be invaluable. In retail, there has been a large push for credit card application because of research showing that customers with a credit card tend to frequent the store more than customers without a credit card. Using the dataset, the department store would be able to see what kind of wiggle room they have to get more customers approved for a credit card and thus, having a more steady cash flow